# A/B test

Importing modules

In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

In [28]:
ab_df = pd.read_csv('ab_data.csv')

In [29]:
ab_df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


In [30]:
ab_df[['group', 'landing_page']].value_counts()

group      landing_page
treatment  new_page        145311
control    old_page        145274
treatment  old_page          1965
control    new_page          1928
Name: count, dtype: int64

В данных ошибка: присутствуют пользователи из контрольной группы, которые использовали новую страницу, и пользователи из экспериментальной группы, которые взаимодействовали с старой страницей.

Очистим данные

In [31]:
df_cleaned = ab_df.loc[(ab_df['group'] == 'control') & (ab_df['landing_page'] == 'old_page') | (ab_df['group'] == 'treatment') & (ab_df['landing_page'] == 'new_page')]
df_cleaned.groupby(['group','landing_page']).count()

,,user_id,timestamp,converted
group,landing_page,,,
control,old_page,145274,145274,145274
treatment,new_page,145311,145311,145311


Теперь проверим на наличие дубликатов

In [32]:
df_cleaned[df_cleaned['user_id'].duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [33]:
df_cleaned[df_cleaned['user_id']==773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


Пользователь 773192 2 раза взаимодействовал с новой страницей и оба раза конверсии не было, можем оставить только одно значение

In [34]:
df_cleaned = df_cleaned.drop_duplicates(subset='user_id', keep="first")

Разобьем данные на A и B группу

In [71]:
a = df_cleaned[df_cleaned['group'] == 'control']
b = df_cleaned[df_cleaned['group'] == 'treatment']

Проведем хи-квадрат тест

In [72]:
a_conv = a.converted.sum()
a_noconv = a.converted.count() - a.converted.sum()
b_conv = b.converted.sum()
b_noconv = b.converted.count() - b.converted.sum()
X = np.array([[a_click, a_noclick], [b_click, b_noclick]])

In [76]:
scipy.stats.chi2_contingency(X, correction=False)[1]

0.18988337448194853

p-value равен 19%, необходимо было 5%, следовательно мы не отвергаем нулевую гипотезу

In [78]:
a_CR = a_conv / (a_conv + a_noconv) 
b_CR = b_conv / (b_conv + b_noconv) 
print(a_CR, b_CR)

0.1203863045004612 0.11880806551510564


Разница и вправду совсем маленькая